In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import cv2
import os
%matplotlib inline

In [2]:
df = pd.read_csv("/content/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
df.isnull().sum()

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [8]:
cat_cols =
for col in df.columns:
  if df[col].dtype == "object":
    print(col , df[col].unique())

Sex ['M' 'F']
ChestPainType ['ATA' 'NAP' 'ASY' 'TA']
RestingECG ['Normal' 'ST' 'LVH']
ExerciseAngina ['N' 'Y']
ST_Slope ['Up' 'Flat' 'Down']


In [9]:
cat_cols2 = ["Sex","ExerciseAngina"]
cat_cols3 = ["ChestPainType","RestingECG","ST_Slope"]

In [11]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
le = LabelEncoder()
df["Sex"] = le.fit_transform(df["Sex"])

In [12]:
df["ExerciseAngina"] = le.fit_transform(df["ExerciseAngina"])

In [21]:
oh_chest = OneHotEncoder()
df["ChestPainType"] = oh_chest.fit_transform(df[["ChestPainType"]]).toarray().astype(int)


In [26]:
df["RestingECG"].value_counts()

,count
RestingECG,
Normal,552
LVH,188
ST,178


In [27]:
df["ST_Slope"].value_counts()

,count
ST_Slope,
Flat,460
Up,395
Down,63


In [28]:
df["RestingECG"] = df["RestingECG"].replace(["Normal","ST","LVH"],[0,1,2])
df["ST_Slope"] = df["ST_Slope"].replace(["Up","Flat","Down"],[0,1,2])

<ipython-input-28-3209d0ff95f3>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["RestingECG"] = df["RestingECG"].replace(["Normal","ST","LVH"],[0,1,2])
<ipython-input-28-3209d0ff95f3>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["ST_Slope"] = df["ST_Slope"].replace(["Up","Flat","Down"],[0,1,2])


In [33]:
df.dtypes

,0
Age,int64
Sex,int64
ChestPainType,int64
RestingBP,int64
Cholesterol,int64
FastingBS,int64
RestingECG,int64
MaxHR,int64
ExerciseAngina,int64
Oldpeak,float64


In [34]:
X = df.drop("HeartDisease",axis=1)
y = df["HeartDisease"]

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)
X

array([[-1.4331398 ,  0.51595242, -1.08413811, ..., -0.8235563 ,
        -0.83243239, -1.05211381],
       [-0.47848359, -1.93816322, -1.08413811, ..., -0.8235563 ,
         0.10566353,  0.59607813],
       [-1.75135854,  0.51595242, -1.08413811, ..., -0.8235563 ,
        -0.83243239, -1.05211381],
       ...,
       [ 0.37009972,  0.51595242,  0.9223917 , ...,  1.21424608,
         0.29328271,  0.59607813],
       [ 0.37009972, -1.93816322, -1.08413811, ..., -0.8235563 ,
        -0.83243239,  0.59607813],
       [-1.64528563,  0.51595242, -1.08413811, ..., -0.8235563 ,
        -0.83243239, -1.05211381]])

In [39]:
X = pd.DataFrame(X,columns=df.columns[:-1])
X

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,-1.433140,0.515952,-1.084138,0.410909,0.825070,-0.551341,-0.749180,1.382928,-0.823556,-0.832432,-1.052114
1,-0.478484,-1.938163,-1.084138,1.491752,-0.171961,-0.551341,-0.749180,0.754157,-0.823556,0.105664,0.596078
2,-1.751359,0.515952,-1.084138,-0.129513,0.770188,-0.551341,0.492241,-1.525138,-0.823556,-0.832432,-1.052114
3,-0.584556,-1.938163,0.922392,0.302825,0.139040,-0.551341,-0.749180,-1.132156,1.214246,0.574711,0.596078
4,0.051881,0.515952,-1.084138,0.951331,-0.034755,-0.551341,-0.749180,-0.581981,-0.823556,-0.832432,-1.052114
...,...,...,...,...,...,...,...,...,...,...,...
913,-0.902775,0.515952,-1.084138,-1.210356,0.596393,-0.551341,-0.749180,-0.188999,-0.823556,0.293283,0.596078
914,1.536902,0.515952,0.922392,0.627078,-0.053049,1.813758,-0.749180,0.164684,-0.823556,2.357094,0.596078
915,0.370100,0.515952,0.922392,-0.129513,-0.620168,-0.551341,-0.749180,-0.857069,1.214246,0.293283,0.596078
916,0.370100,-1.938163,-1.084138,-0.129513,0.340275,-0.551341,1.733661,1.461525,-0.823556,-0.832432,0.596078


In [40]:
from sklearn.model_selection  import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [41]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [42]:
y_pred = knn.predict(X_test)

In [44]:
from sklearn.metrics import confusion_matrix , classification_report

cm = confusion_matrix(y_test,y_pred)
cm

array([[72, 10],
       [ 9, 93]])

In [46]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.88      0.88        82
           1       0.90      0.91      0.91       102

    accuracy                           0.90       184
   macro avg       0.90      0.89      0.90       184
weighted avg       0.90      0.90      0.90       184

